In [9]:
import pandas as pd
import re

def answer_one():
    energy = pd.read_excel("Energy_Indicators.xls", skiprows=17, nrows=227, usecols="C:F", na_values="...")
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
    
    energy['Energy Supply'] *= 1_000_000

    energy['Country'] = energy['Country'].apply(lambda x: re.sub(r'\d+|\s*\(.*?\)', '', x))

    energy_country_rename_map = {
        'Republic of Korea': 'South Korea',
        'United States of America': 'United States',
        'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom',
        'China, Hong Kong Special Administrative Region': 'Hong Kong',
        'Bolivia (Plurinational State of)': 'Bolivia',
        'Switzerland': 'Switzerland'
    }
    energy['Country'] = energy['Country'].replace(energy_country_rename_map)

    GDP = pd.read_csv("world_bank.csv", skiprows=4)
    GDP = GDP.rename(columns={'Country Name': 'Country'})
    
    GDP_country_rename_map = {
        'Korea, Rep.': 'South Korea',
        'Iran, Islamic Rep.': 'Iran',
        'Hong Kong SAR, China': 'Hong Kong'
    }
    GDP['Country'] = GDP['Country'].replace(GDP_country_rename_map)

    years = [str(year) for year in range(2006, 2016)]
    GDP = GDP[['Country'] + years]

    ScimEn = pd.read_excel('scimagojr country rank 1996-2023.xlsx')

    merged_df = ScimEn.merge(energy, on='Country', how='left') \
                      .merge(GDP, on='Country', how='left')

    merged_df = merged_df[merged_df['Rank'] <= 15]

    merged_df.set_index('Country', inplace=True)

    columns = ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
               'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita',
               '% Renewable'] + years
    merged_df = merged_df[columns]

    return merged_df

answer_one()



,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,402188,400158,5077232,3511673,12.62,343,1.271910e+11,93.0,19.754910,2.752119e+12,3.550328e+12,4.594337e+12,5.101691e+12,6.087192e+12,7.551546e+12,8.532185e+12,9.570471e+12,1.047562e+13,1.106157e+13
United States,2,208042,203356,3540288,1004877,17.02,458,9.083800e+10,286.0,11.570980,1.381558e+13,1.447423e+13,1.476986e+13,1.447807e+13,1.504897e+13,1.559973e+13,1.625397e+13,1.688068e+13,1.760814e+13,1.829502e+13
India,3,81853,79757,968523,369256,11.83,237,3.319500e+10,26.0,14.969080,9.402599e+11,1.216736e+12,1.198895e+12,1.341888e+12,1.675616e+12,1.823052e+12,1.827638e+12,1.856722e+12,2.039126e+12,2.103588e+12
Japan,4,58342,57735,717946,154848,12.31,228,1.898400e+10,149.0,10.232820,4.601663e+12,4.579751e+12,5.106679e+12,5.289493e+12,5.759072e+12,6.233147e+12,6.272363e+12,5.212328e+12,4.896994e+12,4.444931e+12
United Kingdom,5,56288,54713,1095371,179764,19.46,293,7.920000e+09,124.0,10.600470,2.708442e+12,3.090510e+12,2.929412e+12,2.412840e+12,2.485483e+12,2.663806e+12,2.707090e+12,2.784854e+12,3.064708e+12,2.927911e+12
Germany,6,50906,49773,777362,160302,15.27,252,1.326100e+10,165.0,17.901530,3.046309e+12,3.484057e+12,3.808786e+12,3.479801e+12,3.468154e+12,3.824829e+12,3.597897e+12,3.808086e+12,3.965801e+12,3.423568e+12
Russian Federation,7,46186,45868,217996,96688,4.72,112,3.070900e+10,214.0,17.288680,9.899321e+11,1.299703e+12,1.660848e+12,1.222646e+12,1.524917e+12,2.045923e+12,2.208294e+12,2.292470e+12,2.059242e+12,1.363482e+12
Canada,8,41209,40390,915491,142691,22.22,284,1.043100e+10,296.0,61.945430,1.319265e+12,1.468820e+12,1.552990e+12,1.374625e+12,1.617343e+12,1.793327e+12,1.828366e+12,1.846597e+12,1.805750e+12,1.556509e+12
Italy,9,38700,36909,639473,147302,16.52,209,6.530000e+09,109.0,33.667230,1.958564e+12,2.222524e+12,2.417508e+12,2.209484e+12,2.144936e+12,2.306974e+12,2.097929e+12,2.153226e+12,2.173256e+12,1.845428e+12


In [10]:
def answer_two():
    Top15 = answer_one()

    avgGDP = pd.Series(Top15[['2006','2007','2008','2009','2010','2011','2012','2013','2014','2015']].mean(axis=1).sort_values(ascending=False))

    return avgGDP

answer_two()

Country
United States         1.572243e+13
China                 6.927707e+12
Japan                 5.239642e+12
Germany               3.590729e+12
United Kingdom        2.777505e+12
France                2.692000e+12
Italy                 2.152983e+12
Brazil                1.988889e+12
Russian Federation    1.666746e+12
Canada                1.616359e+12
India                 1.602352e+12
Spain                 1.406644e+12
South Korea           1.221328e+12
Australia             1.207997e+12
Iran                  4.567516e+11
dtype: float64

In [11]:
def answer_three():
    Top15 = answer_one()
    avgGDP = answer_two()

    temp = Top15.loc[avgGDP.index[5]]

    return temp['2015'] - temp['2006']

    
answer_three()


np.float64(124621907951.68018)

In [12]:
def answer_four():
    Top15 = answer_one()

    temp = (Top15['Self-citations'] / Top15['Citations']).sort_values(ascending=False)

    return tuple(zip(temp.index, temp))[0]

answer_four()


('China', 0.6916510807463594)

In [13]:
def answer_five():
    Top15 = answer_one()

    temp = (Top15['Energy Supply'] / Top15['Energy Supply per Capita']).sort_values(ascending=False)

    return str(temp.index[2])

answer_five()

'United States'

In [14]:
def answer_six():
    Top15 = answer_one()

    Top15['Citable documents per person'] = Top15['Citable documents'] / (Top15['Energy Supply'] / Top15['Energy Supply per Capita'])

    return Top15['Citable documents per person'].corr(Top15['Energy Supply per Capita'])


answer_six()

np.float64(0.7019601723071188)

In [15]:
def answer_seven():
    Top15 = answer_one()

    Top15['Estimated Population'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']

    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'
    }

    Top15['Continent'] = Top15.index.to_series().map(ContinentDict)
    
    result = Top15.groupby('Continent')['Estimated Population'].agg(['size', 'sum', 'mean', 'std'])
    return result

answer_seven()

,size,sum,mean,std
Continent,,,,
Asia,5,2.898666e+09,5.797333e+08,6.790979e+08
Australia,1,2.331602e+07,2.331602e+07,NaN
Europe,6,4.579297e+08,7.632161e+07,3.464767e+07
North America,2,3.528552e+08,1.764276e+08,1.996696e+08
South America,1,2.059153e+08,2.059153e+08,NaN
